This Python script is designed to interact with a MySQL database. It creates a table, inserts fake data into it, retrieves the data, and then drops the table. Here's a detailed breakdown:

1. **Environment Variables**: The script uses the `dotenv` module to load environment variables from a `.env` file. These variables are used to configure the connection to the MySQL database.

2. **Database Connection**: The script connects to the MySQL database using the `mysql.connector` module. The connection parameters are taken from the environment variables.

3. **Table Creation**: The script creates a new table named `users` in the database. The table has three columns: `id`, `name`, and `email`.

4. **Data Insertion**: The script uses the `Faker` library to generate fake data for testing purposes. It inserts 10 fake users into the `users` table.

5. **Data Retrieval**: The script retrieves all the data from the `users` table and checks that the number of rows matches the number of users it inserted.

6. **Table Deletion**: Finally, the script drops the `users` table and closes the connection to the database.

This script is a good starting point for anyone who needs to write a Python script that interacts with a MySQL database. It demonstrates how to perform basic database operations in Python, and how to use environment variables to configure a database connection.

In [ ]:
import mysql.connector
from faker import Faker
import os
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

DB_CONFIG = {
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT'),
    'database': os.getenv('DB_NAME')
}

CREATE_TABLE_QUERY = "CREATE TABLE `users` ( `id` INT(2) NOT NULL AUTO_INCREMENT , `name` VARCHAR(100) NOT NULL , `email` VARCHAR(50) NOT NULL , PRIMARY KEY (`id`)) ENGINE = InnoDB;"
INSERT_QUERY = "INSERT INTO users (name, email) VALUES (%s, %s)"
SELECT_QUERY = "SELECT * FROM users"
UPDATE_QUERY = "UPDATE users SET name = %s WHERE email = %s"
DELETE_QUERY = "DELETE FROM users WHERE email = %s"
DROP_TABLE_QUERY = "DROP TABLE IF EXISTS `users`"

def create_and_read():
    """Demonstrate CREATE and READ operations with secure parameterized queries"""
    cnx = mysql.connector.connect(**DB_CONFIG)
    cursor = cnx.cursor()
    
    try:
        # Create table
        cursor.execute(DROP_TABLE_QUERY)  # Clean start
        cursor.execute(CREATE_TABLE_QUERY)
        print("✅ Table created successfully")
        
        # Insert fake data using parameterized queries
        fake = Faker()
        total_test_users = 5
        
        for i in range(total_test_users):
            data = (fake.name(), fake.email())
            cursor.execute(INSERT_QUERY, data)
            print(f"➕ Inserted user {i+1}: {data[0]} ({data[1]})")
        
        cnx.commit()
        
        # Read and verify data
        cursor.execute(SELECT_QUERY)
        rows = cursor.fetchall()
        print(f"\n📊 Retrieved {len(rows)} users from database:")
        for row in rows:
            print(f"   ID: {row[0]}, Name: {row[1]}, Email: {row[2]}")
        
        assert len(rows) == total_test_users, f"Expected {total_test_users} users, got {len(rows)}"
        print(f"\n✅ Test passed: {total_test_users} users created and retrieved successfully")
        
    except Exception as e:
        print(f"❌ Error: {e}")
    finally:
        cursor.execute(DROP_TABLE_QUERY)
        cnx.close()
        print("🧹 Cleanup completed")

def test_update_and_delete():
    """Demonstrate UPDATE and DELETE operations"""
    cnx = mysql.connector.connect(**DB_CONFIG)
    cursor = cnx.cursor()
    
    try:
        # Create table and insert test data
        cursor.execute(DROP_TABLE_QUERY)
        cursor.execute(CREATE_TABLE_QUERY)
        
        name = "Test User"
        email = "test@example.com"
        updated_name = "Updated Test User"
        
        cursor.execute(INSERT_QUERY, (name, email))
        cnx.commit()
        print(f"➕ Inserted: {name} ({email})")
        
        # Test UPDATE
        cursor.execute(UPDATE_QUERY, (updated_name, email))
        cnx.commit()
        
        cursor.execute("SELECT name FROM users WHERE email = %s", (email,))
        result = cursor.fetchone()
        print(f"✏️  Updated name to: {result[0]}")
        
        # Test DELETE
        cursor.execute(DELETE_QUERY, (email,))
        cnx.commit()
        
        cursor.execute(SELECT_QUERY)
        remaining_users = cursor.fetchall()
        print(f"🗑️  Deleted user. Remaining users: {len(remaining_users)}")
        
        assert len(remaining_users) == 0, "User should be deleted"
        print("✅ Update and delete operations successful")
        
    except Exception as e:
        print(f"❌ Error: {e}")
    finally:
        cursor.execute(DROP_TABLE_QUERY)
        cnx.close()
        print("🧹 Cleanup completed")

# Run the demonstrations
print("🚀 Starting database operations demonstration...\n")
print("=" * 50)
print("TEST 1: CREATE and READ Operations")
print("=" * 50)
create_and_read()

print("\n" + "=" * 50)
print("TEST 2: UPDATE and DELETE Operations")
print("=" * 50)
test_update_and_delete()

print("\n🎉 All database operations completed successfully!")